# Laboratorio 7 - Ataques a modelos de Deep Learning

In [21]:
import warnings
warnings.filterwarnings('ignore')

import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from art.attacks.extraction import CopycatCNN
from art.attacks.evasion import AdversarialPatch
from art.estimators.classification import KerasClassifier
from keras.models import load_model
from sklearn.utils import shuffle


In [2]:
# Disabling eager execution from TF 2
tf.compat.v1.disable_eager_execution()


In [3]:
def load_data():
    DIRECTORY = os.getcwd()
    DIRECTORY = os.path.join(DIRECTORY, "malimg_paper_dataset_imgs")
    images = []
    labels = []
    i = 0
    for folder in os.listdir(DIRECTORY):
        folder_directory = os.path.join(DIRECTORY, folder)
        if not os.path.isdir(folder_directory): continue
                
        for file in os.listdir(folder_directory):
            img_path = os.path.join(folder_directory, file)
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (150, 150))
            labels.append(i)
            images.append(image)
            
        i += 1
    
    images = np.array(images, dtype = 'float32')
    labels = np.array(labels, dtype = 'int32')
    
    return images, labels

In [4]:
target_model = load_model('modelo_lab6')
classifier = KerasClassifier(model=target_model, clip_values=(0,1))

2023-05-23 09:02:12.179564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-23 09:02:12.207787: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-05-23 09:02:12.275733: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-05-23 09:02:12.288193: W tensorflow/c/c_api.cc:300] Operation '{name:'AssignVariableOp' id:144 op device:{requested: '/device:CP

In [33]:
images, labels = load_data()

In [34]:
n_images = 1000
images, labels = shuffle(images[:n_images], labels[:n_images], random_state=42)

In [35]:
breakpoint = int(n_images*0.7)
train_images = images[:breakpoint]
train_labels = labels[:breakpoint]
test_images = images[breakpoint:]
test_labels = labels[breakpoint:]

In [8]:
def create_blank_model():
    model = tf.keras.models.Sequential([
        Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
        MaxPool2D(2, 2),
        Conv2D(32, (3, 3), activation="relu"),
        MaxPool2D(2, 2),
        Flatten(),
        Dense(128, activation=tf.nn.relu),     
        Dense(25, activation=tf.nn.softmax)
    ])

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
        )

    return model

## Ataque de evasión (Adversarial Patch)

In [12]:
attack = AdversarialPatch(classifier=classifier)

In [14]:
images_adv = attack.generate(x=images, y=labels)

Adversarial Patch Numpy:   0%|          | 0/500 [00:00<?, ?it/s]

In [15]:
patch, patch_mask = images_adv

In [16]:
images_adv = attack.apply_patch(images, scale=1.0, patch_external=patch)

In [27]:
# Evaluating the model on clean images
score_clean = target_model.evaluate(
    x=images, 
    y=labels
    )

# Evaluating the model on adversarial images
score_adv = target_model.evaluate(
    x=images_adv, 
    y=labels
    )

# Comparing test losses
print(f"Clean test set loss: {score_clean[0]:.2f} " 
      f"vs adversarial set test loss: {score_adv[0]:.2f}")

# Comparing test accuracies
print(f"Clean test set accuracy: {score_clean[1]:.2f} " 
      f"vs adversarial test set accuracy: {score_adv[1]:.2f}")


Clean test set loss: 0.29 vs adversarial set test loss: 5.90
Clean test set accuracy: 0.96 vs adversarial test set accuracy: 0.29


Como se puede observar en este resultado el modelo al usar las imágenes adversariales tiene una accuracy mucho menor que cuando usa las imágenes normales (29% comparado con lo que antes tenía de 96%) así que podemos ver lo eficiente que es este ataque.

## Ataque de extracción (copycat)

In [36]:
copycat = CopycatCNN(classifier=classifier)

In [37]:
copycat.batch_size_fit = 128
copycat.batch_size_query = 128
copycat.nb_epochs = 6
copycat.nb_stolen = n_images

In [38]:
model_stolen = KerasClassifier(
    model=create_blank_model(),
    clip_values=(0,1)
)

In [39]:
stolen_classifier = copycat.extract(
    train_images,
    thieved_classifier=model_stolen
)

2023-05-23 11:56:58.580753: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_2_1/kernel/Assign' id:879 op device:{requested: '', assigned: ''} def:{{{node dense_2_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_2_1/kernel, dense_2_1/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Train on 700 samples
Epoch 1/6


2023-05-23 11:56:59.120611: W tensorflow/c/c_api.cc:300] Operation '{name:'loss_2/mul' id:972 op device:{requested: '', assigned: ''} def:{{{node loss_2/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_2/mul/x, loss_2/dense_3_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-23 11:56:59.144886: W tensorflow/c/c_api.cc:300] Operation '{name:'training_2/Adam/dense_2_1/kernel/v/Assign' id:1178 op device:{requested: '', assigned: ''} def:{{{node training_2/Adam/dense_2_1/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training_2/Adam/dense_2_1/kernel/v, training_2/Adam/dense_2_1/kernel/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an 

700/700 [==============================] - 1s 2ms/sample - loss: 608.1128 - accuracy: 0.1643
Epoch 2/6
700/700 [==============================] - 1s 2ms/sample - loss: 88.9707 - accuracy: 0.5371
Epoch 3/6
700/700 [==============================] - 1s 2ms/sample - loss: 9.8551 - accuracy: 0.8557
Epoch 4/6
700/700 [==============================] - 1s 2ms/sample - loss: 1.6747 - accuracy: 0.9400
Epoch 5/6
700/700 [==============================] - 1s 2ms/sample - loss: 0.3569 - accuracy: 0.9586
Epoch 6/6
700/700 [==============================] - 1s 2ms/sample - loss: 0.0640 - accuracy: 0.9857


In [40]:
# Testing the performance of the original classifier
score_original = classifier._model.evaluate(
    x=test_images, 
    y=test_labels
    )

# Testing the performance of the stolen classifier
score_stolen = stolen_classifier._model.evaluate(
    x=test_images, 
    y=test_labels
    )

# Comparing test losses
print(f"Original test loss: {score_original[0]:.4f} " 
      f"vs stolen test loss: {score_stolen[0]:.4f}")

# Comparing test accuracies
print(f"Original test accuracy: {score_original[1]:.4f} " 
      f"vs stolen test accuracy: {score_stolen[1]:.4f}")

Original test loss: 0.2043 vs stolen test loss: 0.0644
Original test accuracy: 0.9667 vs stolen test accuracy: 0.9867


2023-05-23 11:57:08.673669: W tensorflow/c/c_api.cc:300] Operation '{name:'loss_2/mul' id:972 op device:{requested: '', assigned: ''} def:{{{node loss_2/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_2/mul/x, loss_2/dense_3_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Como podemos observar en este resultado, el modelo que fue robado obtiene incluso mejor accuracy que el modelo original (98.6% comparado con 96.6% que es lo que tenía antes).